In [1]:
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
import os
from photutils.aperture import CircularAnnulus, CircularAperture, aperture_photometry, ApertureStats
from astropy.stats import SigmaClip

#from jwst.residual_fringe.utils import fit_residual_fringes_1d as rf1d

import glob
from photutils.aperture import aperture_photometry, CircularAperture

#files_ch1 = sorted(glob.glob("/aux/vcompute3a/tamojeet/workspace/jwst_new/Calints Trials/S3D/*ch1*s3d.fits"))
files_ch1 = sorted(glob.glob("6_2/*ch1*s3d.fits"))
files_ch2 = sorted(glob.glob("6_2/*ch2*s3d.fits"))
files_ch3 = sorted(glob.glob("6_2/*ch3*s3d.fits"))
files_ch4 = sorted(glob.glob("6_2/*ch4*s3d.fits"))
#files_ch1
# files_ch1, files_ch2, files_ch3, files_ch4

### Code for differential image making. To get the times array, run any one of the channel-lightcurve making cells below these _first_.

#### _Note_: The files are not in order because alphabetically 10 (or 11 or 12. . .) comes before 2 (or 3/4/5. . . ) - my bad naming! To sort them in correct order, we use the goodorder cell below which sorts by time

In [4]:
files = files_ch4
times = np.zeros(85)
for i in range(85):
    ff = fits.open(files[i])
    data = np.nanmean(ff[1].data, axis = 0)
    h = ff[0].header
    if h["DATE-OBS"] == '2024-04-06':
        t = h['EXPSTART']-60400 + (h['INTSTART']+h['INTEND'])/2*h['EFFINTTM']/86400
        times[i] = t

goodorder = np.argsort(times)
goodorder

/tmp/ipykernel_484/1910202042.py:5: RuntimeWarning: Mean of empty slice
  data = np.nanmean(ff[1].data, axis = 0)


array([ 0,  9, 10, 11, 12, 13, 14, 15, 16, 17,  1,  2,  3,  4,  5,  6,  7,
        8, 18, 27, 28, 29, 30, 31, 32, 33, 34, 35, 19, 20, 21, 22, 23, 24,
       25, 26, 36, 45, 46, 47, 48, 49, 50, 51, 52, 53, 37, 38, 39, 40, 41,
       42, 43, 44, 54, 63, 64, 65, 66, 67, 68, 69, 70, 71, 55, 56, 57, 58,
       59, 60, 61, 62, 72, 76, 77, 78, 79, 80, 81, 82, 83, 84, 73, 74, 75])

In [219]:
stack1 = np.array([np.nanmedian(fits.open(files_ch1[goodorder[i]])[1].data, axis=0) for i in range(85)])

for i in range(stack1.shape[1]):
    for j in range(stack1.shape[2]):
        stack1[:,i,j] -= np.nanmedian(stack1[:,i,j])
        
hh = fits.open(files_ch1[25])[1].header
newd = fits.PrimaryHDU(data=stack1, header=hh)
newd.writeto('Integration_stack/ch1-6apr.fits',overwrite=True)

/home/tamojeet/.local/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1095: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)
